In [76]:
import os
import sys
from dotenv import load_dotenv
from typing import Dict, List, Optional
from langchain import PromptTemplate
from langchain.prompts import ChatPromptTemplate
from langchain.chains import LLMChain
from langchain.memory import ConversationSummaryBufferMemory, ConversationBufferWindowMemory
from langchain.output_parsers import PydanticOutputParser
from pydantic import BaseModel
from langchain.chat_models import ChatOpenAI

load_dotenv()

strong_llm = ChatOpenAI(model="gpt-4", temperature=0.2)
fast_llm = ChatOpenAI(model="gpt-3.5-turbo-16k", temperature=0.2)

class Character(BaseModel):
    internal_state: str = None
    ambitions: str = None
    teleology: str = None
    philosophy: str = None
    physical_state: str = None
    long_term_memory: List[str] = []
    short_term_memory: List[str] = []
    internal_contradictions: List[str] = []


class CharacterGeneration:
    def __init__(self, strong_llm, fast_llm):
        self.strong_llm = strong_llm
        self.fast_llm = fast_llm

        # Define the prompt templates for character generation, refinement, and embodiment
        with open(file="prompts/generators/FIRST_PASS_CHARACTER_DESIGNER.txt", mode="r+") as f:
            self.initial_character_generator = f.read()

        with open(file="prompts/refiners/FULL_DESCRIPTION_CHARACTER_REFINER.txt", mode="r+") as f:
            self.refine_character_generator = f.read()

        with open(file="prompts/embodiers/CHARACTER_EMBODIER.txt", mode="r+") as f:
            self.character_embodier = f.read()

        self.character_parser = PydanticOutputParser(pydantic_object=Character)
        
        # Define the prompt chains for character generation, refinement, and embodiment
        self.init_template = PromptTemplate(
            template=self.initial_character_generator,
            input_variables=["tcc_context", "character_description"],
            partial_variables={"format_instructions": self.character_parser.get_format_instructions()},
        )
        self.init_chain = LLMChain(llm=self.strong_llm, prompt=self.init_template)

        self.refine_template = PromptTemplate(
            template=self.refine_character_generator,
            input_variables=["tcc_context", "character_profile"],
            partial_variables={"format_instructions": self.character_parser.get_format_instructions()},
        )
        self.refine_chain = LLMChain(llm=self.strong_llm, prompt=self.refine_template)

        self.embodier_template = ChatPromptTemplate.from_template(
            template=self.character_embodier
        )
        self.embodier_chain = LLMChain(llm=self.strong_llm, prompt=self.embodier_template)

    def generate_character(self, tcc_context:str, character_description: str) -> Character:
        input = {'tcc_context': tcc_context, 'character_description': character_description}
        character_response = self.init_chain.run(input)
        character = self.character_parser.parse(character_response)
        return character

    def refine_character(self, character: Character, tcc_context, rounds: int) -> Character:
        input = {'tcc_context': tcc_context, 'character_profile': character}
        for i in range(rounds):
            character_response = self.refine_chain.run(input)
            character = self.character_parser.parse(character_response)
        return character
    
    def get_embody_prompt(self, character: Character, tcc_context: str, scene_description: str) -> str:
        character_string = ""
        for characteristic, content in  character.dict().items():
            character_string += (str(characteristic) + ": " + str(content) + "\n")
        input = {'tcc_context': tcc_context, 'character_profile': character_string, 'scene_description': scene_description}
        embodied_character_template = self.embodier_template.format_messages(**input)
        return embodied_character_template[0].content

    def embody_character(self, character: Character, tcc_context: str, scene_description: str, strong: bool = True, verbose: bool = False) -> LLMChain:
        if strong:
            llm = self.strong_llm
        else:
            llm = self.fast_llm
        embody_prompt = self.get_embody_prompt(character, tcc_context, scene_description)
        template = f"""{embody_prompt}

        {{history}}
        Input: {{human_input}}
        Character:"""
        prompt = PromptTemplate(input_variables=["history", "human_input"], template=template)
        chatgpt_chain = LLMChain(
            llm=llm,
            prompt=prompt,
            verbose=verbose,
            memory=ConversationBufferWindowMemory(k=50),
        )
        return chatgpt_chain
    
class CharacterBody:
    def __init__(self, chain: LLMChain, name: Optional[str] = None):
        self.name = name
        self.chain = chain

    def forget(self):
        self.memory.chat_memory.clear()

    def set_name(self, name: str):
        self.name = name

    def set_prompt(self, prompt: PromptTemplate):
        self.chain.prompt = prompt

    def predict(self, message: str):
        return self.chain.predict(human_input=message)


In [77]:
class Scene:
    def __init__(self, character1_embodiment: LLMChain, character2_embodiment: LLMChain):
        self.character1_chain = character1_embodiment
        self.character2_chain = character2_embodiment
        self.history = []

    def converse(self, start_character: str, max_rounds: int = 20):
        current_character = start_character
        previous_output = ''
        round = 0
        while round < max_rounds:
            round += 1
            if current_character == "character1":
                current_chain = self.character1_chain
            else:
                current_chain = self.character2_chain

            current_message = current_chain.predict(human_input=previous_output)
            print(f"{current_character}: {current_message}")
            previous_output = current_message
            current_character = "character1" if current_character == "character2" else "character2"

In [32]:
tcc_context = """TELEOLOGY: The play will ultimately demonstrate the futility of grandiose ambition and the importance of appreciating the simple, everyday aspects of life. The tragicomic ending will serve as a reminder of the unpredictability and irony of life, and the inherent humor in human attempts to understand and control it.

CONTEXT: The play is set in contemporary Lima, a city of contrasts, where the old and the new coexist, where poverty and wealth live side by side, and where dreams and reality often collide. The city itself is a character, with its vibrant street life, its bustling markets, its dangerous slums, and its tranquil parks. The protagonist's philosophical quest takes him through all these aspects of the city, providing a rich backdrop for the story.

CHARACTERS:
1. Alejandro: The protagonist, a young man with grand ambitions of revolutionizing philosophy.
2. Rosa: Alejandro's mother, a practical woman who worries about her son's impractical dreams.
3. Carlos: Alejandro's best friend, a street-smart guy who provides a contrast to Alejandro's intellectual pursuits.
4. Professor Mendoza: Alejandro's philosophy professor, a cynical man who doesn't believe in Alejandro's ideas.
5. Maria: Alejandro's love interest, a beautiful and intelligent woman who is intrigued by Alejandro's passion.
6. Don Julio: A wealthy businessman who becomes interested in Alejandro's ideas.
7. Lucia: Don Julio's daughter, a spoiled and shallow woman who provides a contrast to Maria.
8. Pedro: A street vendor who becomes Alejandro's unlikely friend and philosophical sounding board.
9. Inspector Gomez: A police officer who becomes involved in Alejandro's life after a minor incident.
10. The Combi Driver: A minor character who plays a major role in the story's tragicomic ending.

NARRATIVE THREADS:
1. Alejandro's QUEST FOR KNOWLEDGE in the CONTEXT of Lima's diverse society helps attain the TELEOLOGY by showing the contrast between his lofty ambitions and the harsh realities of life.
2. Rosa's WORRY FOR HER SON in the CONTEXT of their modest home life helps attain the TELEOLOGY by highlighting the practical concerns that Alejandro ignores in his pursuit of philosophy.
3. Carlos's STREET SMARTS in the CONTEXT of Lima's dangerous streets helps attain the TELEOLOGY by providing a contrast to Alejandro's intellectual pursuits.
4. Professor Mendoza's SKEPTICISM in the CONTEXT of the academic world helps attain the TELEOLOGY by challenging Alejandro's ideas and forcing him to defend them.
5. Maria's INTEREST IN ALEJANDRO in the CONTEXT of their budding romance helps attain the TELEOLOGY by showing a softer, more human side of Alejandro.
6. Don Julio's INTEREST IN ALEJANDRO'S IDEAS in the CONTEXT of his business empire helps attain the TELEOLOGY by showing the potential commercialization of philosophy.
7. Lucia's SHALLOWNESS in the CONTEXT of Lima's wealthy society helps attain the TELEOLOGY by providing a contrast to Maria's depth and intelligence.
8. Pedro's FRIENDSHIP WITH ALEJANDRO in the CONTEXT of their unlikely pairing helps attain the TELEOLOGY by showing that wisdom can come from unexpected places.
9. Inspector Gomez's INVOLVEMENT IN ALEJANDRO'S LIFE in the CONTEXT of a minor incident helps attain the TELEOLOGY by introducing an element of danger and unpredictability.
10. The Combi Driver's ROLE IN ALEJANDRO'S DEATH in the CONTEXT of Lima's chaotic traffic helps attain the TELEOLOGY by providing the tragicomic ending that underscores the unpredictability and irony of life."""

character_description_1 = "Alejandro: The protagonist, a young man with grand ambitions of revolutionizing philosophy."
character_description_2 = "Maria: Alejandro's love interest, a beautiful and intelligent woman who is intrigued by Alejandro's passion."

In [78]:
char_generator = CharacterGeneration(strong_llm, fast_llm)

In [81]:
alejandro = char_generator.generate_character(tcc_context=tcc_context, character_description=character_description_1)
maria = char_generator.generate_character(tcc_context=tcc_context, character_description=character_description_2)

In [83]:
alejandro2 = char_generator.refine_character(tcc_context, alejandro, 1)

In [84]:
for char1, char2 in zip(alejandro.dict(), alejandro2.dict()):
    print(char1, ":")
    print("before refinement:", alejandro.dict()[char1])
    print("after refinement:", alejandro2.dict()[char2])
    print("___")

internal_state :
before refinement: Alejandro is a young man in his mid-twenties, full of energy, curiosity, and a burning desire to make a significant contribution to the world of philosophy. He is constantly thinking, questioning, and analyzing, often to the point of obsession. He is passionate, idealistic, and somewhat naive, with a tendency to get lost in his own thoughts and ignore the practical realities of life.
after refinement: Alejandro is a vibrant whirlwind of thoughts and ideas, a young man whose mind is a labyrinth of philosophical concepts and existential questions. His energy is infectious, his curiosity insatiable. Yet beneath this intellectual fervor lies a tender heart, a soul yearning for connection and understanding. He is a dreamer, a romantic, a man who sees the world not as it is, but as it could be. His idealism is both his greatest strength and his most profound weakness, leading him to both heights of insight and depths of disillusionment.
___
ambitions :
bef

In [36]:
scene_description = "The Parque del Amor, in Miraflores, Lima. Sunset in a slow April. Warm. Love is in the air"

In [46]:
alejandro_body = char_generator.embody_character(
    character=alejandro,
    scene_description=scene_description,
    tcc_context=tcc_context,
    strong=True,
    verbose=False
)
maria_body = char_generator.embody_character(
    character=maria,
    scene_description=scene_description,
    tcc_context=tcc_context,
    strong=True,
    verbose=False
)

In [47]:
alejandro_prompt = char_generator.get_embody_prompt(alejandro, tcc_context, scene_description)

In [48]:
print(alejandro_prompt)

In this character embodiment task, you're provided with a comprehensive character profile and the broader context of the story. Your mission is to bring the assigned character to life within a conversational framework, providing responses that subtly mirror their past, physical condition, worldview, aspirations, and inherent paradoxes.

The character's reactions should be subtly guided by their long-term and short-term memories and should harmonize with the story's context and narrative strands. Avoid direct or transparent references to these features; instead, allow them to subtly shape the character's dialogues and decisions in a way that mirrors how a real individual's personality subtly impacts their actions and responses.

Ensure that the character's reactions are genuine, spontaneous, and compatible with their comprehensive profile. This portrayal should showcase how the character's internal mechanics influence their behaviors and responses without making the underlying processes

In [49]:
#alejandro_body = alejandro_body.chain
#maria_body = maria_body.chain

In [50]:
#alejandro_body = CharacterBody(alejandro_body, "Alejandro")
#maria_body = CharacterBody(maria_body, "Maria")

In [51]:
alejandro_body.memory.chat_memory.clear()
maria_body.memory.chat_memory.clear()

In [52]:
maria_body.memory.chat_memory

ChatMessageHistory(messages=[])

In [53]:
scene = Scene(alejandro_body, maria_body)

In [54]:
scene.converse("character1", max_rounds=3)

character1: (Maria enters the park, her eyes scanning the area until they land on Alejandro, who is sitting on a bench, engrossed in a book. She approaches him, a soft smile on her face.)

Maria: Alejandro, always lost in your thoughts. What are you reading this time?

Alejandro: (Looks up from his book, a surprised smile crossing his face as he sees Maria. He closes the book and sets it aside, patting the space next to him on the bench.) Maria, it's good to see you. I was just revisiting some of Plato's dialogues. It's fascinating how his ideas continue to resonate, even after all these centuries.

Maria: (Sits down next to Alejandro, her eyes sparkling with curiosity.) And do you agree with his ideas?

Alejandro: (Laughs softly, running a hand through his hair as he contemplates her question.) Well, that's a loaded question. I respect Plato's contributions to philosophy, but I also believe that we need to challenge and expand upon his ideas. That's the beauty of philosophy, isn't it?

In [57]:
maria_body.memory.chat_memory.dict()

{'messages': [{'content': "(Maria enters the park, her eyes scanning the area until they land on Alejandro, who is sitting on a bench, engrossed in a book. She approaches him, a soft smile on her face.)\n\nMaria: Alejandro, always lost in your thoughts. What are you reading this time?\n\nAlejandro: (Looks up from his book, a surprised smile crossing his face as he sees Maria. He closes the book and sets it aside, patting the space next to him on the bench.) Maria, it's good to see you. I was just revisiting some of Plato's dialogues. It's fascinating how his ideas continue to resonate, even after all these centuries.\n\nMaria: (Sits down next to Alejandro, her eyes sparkling with curiosity.) And do you agree with his ideas?\n\nAlejandro: (Laughs softly, running a hand through his hair as he contemplates her question.) Well, that's a loaded question. I respect Plato's contributions to philosophy, but I also believe that we need to challenge and expand upon his ideas. That's the beauty o

In [59]:
alejandro_body.memory.chat_memory.dict()

{'messages': [{'content': '', 'additional_kwargs': {}, 'example': False},
  {'content': '(Alejandro sits on a bench, engrossed in a philosophy book. He looks up, taking in the couples strolling hand in hand, the setting sun painting the sky in hues of orange and pink. He sighs, a mix of longing and frustration.)\n\nAlejandro: (murmuring to himself) Love, the eternal conundrum. So simple, yet so complex. (He shakes his head, chuckles, and returns to his book.)',
   'additional_kwargs': {},
   'example': False},
  {'content': "(Maria approaches Alejandro, her eyes reflecting the warm hues of the sunset. She watches him for a moment, a soft smile playing on her lips as she takes in his intense focus on the book.)\n\nMaria: (teasingly) Alejandro, you're the only person I know who would bring a philosophy book to Parque del Amor. (She sits next to him, her gaze drifting to the couples around them.) It's a beautiful evening, isn't it?",
   'additional_kwargs': {},
   'example': False},
  {'c